# LSTM Models with 3 different languages

### Create the configuration for the experiment

All the languages with use the same model configuration for comparing performances across them

In [1]:
%pip install conllu

Note: you may need to restart the kernel to use updated packages.


In [2]:
import keras

keras.utils.set_random_seed(50)

/opt/conda/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibility violations in the next r

In [3]:
from data.preprocessor import DataPreprocessor, DataPreprocessorConfig
from trainer.trainer import TrainerConfig, Trainer
from models.base_model import ModelConfig

preprocessor_config = DataPreprocessorConfig(
    padding_type="post",
    truncation_type="post",
    remove_long_sentences=True,
    max_sequence_length=100,
)

training_config = TrainerConfig(
    epochs=20,
    batch_size=64,
    early_stopping_patience=3,
    learning_rate=1e-3,
    model_dir="saved_models",
    save_best_only=True,
)

model_config = ModelConfig(
    embedding_dim=80,
    lstm_units=128,
    bidirectional=True,
    dropout_rate=0.3,
    training_config=training_config,
)

## English

### Preprocesses the data

In [4]:
from utils import load_data

train_data_en, dev_data_en, test_data_en = load_data("english")

In [5]:
preprocessor_en = DataPreprocessor(preprocessor_config)

In [6]:
X_train_en, y_train_en = preprocessor.process_data_to_pad_sequences(
    train_data_en, is_train_dataset=True
)
X_dev_en, y_dev_en = preprocessor.process_data_to_pad_sequences(
    dev_data, is_train_dataset=False
)
X_test_en, y_test_en = preprocessor.process_data_to_pad_sequences(
    test_data, is_train_dataset=False
)

Sample sentence: Al - Zaman : American forces killed Shaikh Abdullah al - Ani , the preacher at the mosque in the town of Qaim , near the Syrian border .
Sample tags: PROPN PUNCT PROPN PUNCT ADJ NOUN VERB PROPN PROPN PROPN PUNCT PROPN PUNCT DET NOUN ADP DET NOUN ADP DET NOUN ADP PROPN PUNCT ADP DET ADJ NOUN PUNCT
Sample vectorized sentence shape: (100,)
Sample vectorized sentence: [  264    16  6152    45   293   709  1150  4860 18886   592    16 18792
     4     3  7335    35     3  7509    10     3   486     8  8930     4
   751     3  2357  1642     2     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
Sample

### Initilize the model with the config

In [7]:
from models.lstm_model import LSTMModel

lstm_en = LSTMModel(
    model_config,
    preprocessor_en.vocab_size,
    preprocessor_en.num_tags,
    preprocessor_config.max_sequence_length,
)
lstm_en.build_model()
lstm_en.compile_model()

print("Model summary:\n")
print(lstm_en.get_model().summary())

Model summary:



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 80)        │     1,574,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 256)       │       214,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 100, 19)        │         4,883 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,792,979 (6.84 MB)

 Trainable params: 1,792,979 (6.84 MB)

 Non-trainable params: 0 (0.00 B)

None


### Training the model

In [8]:
# Initialize trainer
trainer_en = Trainer(training_config, lstm_en, preprocessor)

In [9]:
# Train the model
print("Training model...\n")
trainer_en.train((X_train_en, y_train_en), (X_dev_en, y_dev_en))
print("Training completed.\n")

Training model...

Epoch 1/20
196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - _masked_accuracy: 0.3955 - loss: 2.0569
Epoch 1: val_loss improved from None to 0.57147, saving model to saved_models/BiLSTM_emb80_lstm128_drop30_ep20_bs64_pat3_lr10.keras
196/196 ━━━━━━━━━━━━━━━━━━━━ 40s 196ms/step - _masked_accuracy: 0.5975 - loss: 1.3587 - val__masked_accuracy: 0.8299 - val_loss: 0.5715
Epoch 2/20
196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - _masked_accuracy: 0.8872 - loss: 0.3990
Epoch 2: val_loss improved from 0.57147 to 0.33703, saving model to saved_models/BiLSTM_emb80_lstm128_drop30_ep20_bs64_pat3_lr10.keras
196/196 ━━━━━━━━━━━━━━━━━━━━ 46s 233ms/step - _masked_accuracy: 0.9123 - loss: 0.3171 - val__masked_accuracy: 0.8975 - val_loss: 0.3370
Epoch 3/20
196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step - _masked_accuracy: 0.9513 - loss: 0.1815
Epoch 3: val_loss improved from 0.33703 to 0.32228, saving model to saved_models/BiLSTM_emb80_lstm128_drop30_ep20_bs64_pat3_lr10.keras
196/196 ━━━━━━━━━━━

### Testing the model

In [10]:
from evaluator.evaluator import Evaluator

evaluator_en = Evaluator(lstm_en, preprocessor_en)

In [11]:
test_metrics_en = evaluator_en.evaluate(X_test_en, y_test_en, "Test")


Test Set Evaluation:
Accuracy: 0.9138

Detailed Classification Report:
              precision    recall  f1-score   support

         ADJ       0.91      0.86      0.88      1788
         ADP       0.94      0.97      0.95      2029
         ADV       0.91      0.89      0.90      1191
         AUX       0.98      0.98      0.98      1543
       CCONJ       0.99      0.99      0.99       736
         DET       0.98      0.99      0.99      1897
        INTJ       0.98      0.70      0.82       121
        NOUN       0.77      0.94      0.85      4123
         NUM       0.90      0.74      0.81       542
        PART       0.97      0.97      0.97       649
        PRON       0.99      0.98      0.99      2165
       PROPN       0.89      0.60      0.71      2075
       PUNCT       1.00      0.99      0.99      3096
       SCONJ       0.92      0.77      0.84       384
         SYM       0.79      0.85      0.82       109
        VERB       0.92      0.93      0.92      2606
         

### Inference on new sentences

In [12]:
from inference.predictor import Predictor

# Create predictor for inference
predictor_en = Predictor(lstm_en.get_model(), preprocessor_en)

In [36]:
test_sentences = {
    "Simple Case": [
        "Today it is cloudy",
        "The quick brown fox jumps over the lazy dog ."
    ],
    "Ambiguity": [
        "The leaves are falling .", # leaves => NOUN
        "He leaves tomorrow .", # leaves => VERB
        "I bought an apple .", # apple => NOUN
        "I work at Apple ." # Apple => PROPN
    ],
    "OOV & Typos": [
        "I googled this supercalifragilisticexpialidocious wrd .",
        "This sentance has twoo mispellings ."
    ]
}

for category, sentences in test_sentences.items():
    print(f"\n--- {category} ---")
    for sentence in sentences:
        predicted_tags = predictor.predict_sentence(sentence)
        print(f"  Sentence: {sentence}")
        print(f"  Tags:     {' '.join(predicted_tags)}")


--- Simple Case ---
  Sentence: Today it is cloudy
  Tags:     NOUN PRON AUX ADJ
  Sentence: The quick brown fox jumps over the lazy dog .
  Tags:     DET ADJ ADJ NOUN VERB ADP DET ADJ NOUN PUNCT

--- Ambiguity ---
  Sentence: The leaves are falling .
  Tags:     DET NOUN AUX VERB PUNCT
  Sentence: He leaves tomorrow .
  Tags:     PRON VERB NOUN PUNCT
  Sentence: I bought an apple .
  Tags:     PRON VERB DET NOUN PUNCT
  Sentence: I work at Apple .
  Tags:     PRON VERB ADP PROPN PUNCT

--- OOV & Typos ---
  Sentence: I googled this supercalifragilisticexpialidocious wrd .
  Tags:     PRON VERB DET NOUN NOUN PUNCT
  Sentence: This sentance has twoo mispellings .
  Tags:     DET NOUN VERB ADJ NOUN PUNCT


## Spanish

In [14]:
from utils import load_data

train_data_es, dev_data_es, test_data_es = load_data("spanish")

In [15]:
preprocessor_es = DataPreprocessor(preprocessor_config)

In [16]:
X_train_es, y_train_es = preprocessor_es.process_data_to_pad_sequences(
    train_data_es, is_train_dataset=True
)
X_dev_es, y_dev_es = preprocessor_es.process_data_to_pad_sequences(
    dev_data_es, is_train_dataset=False
)
X_test_es, y_test_es = preprocessor_es.process_data_to_pad_sequences(
    test_data_es, is_train_dataset=False
)

Sample sentence: Además se le pediría a las empresas interesadas en prestar el servicio que se hagan cargo de la señalización y la cartelería que contiene información para los usuarios .
Sample tags: ADV PRON PRON VERB ADP DET NOUN ADJ ADP VERB DET NOUN SCONJ PRON VERB NOUN ADP DET NOUN CCONJ DET NOUN SCONJ VERB NOUN ADP DET NOUN PUNCT
Sample vectorized sentence shape: (100,)
Sample vectorized sentence: [  168    12    37 22916     9    17   656 25241     7  9426     4   247
    10    12  5355   625     2     6  9162     8     6 30478    10  1304
   614    23    11  1380     5     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0   

### Initilize the model with the config

In [17]:
from models.lstm_model import LSTMModel

es_lstm = LSTMModel(
    model_config,
    preprocessor_es.vocab_size,
    preprocessor_es.num_tags,
    preprocessor_config.max_sequence_length,
)
es_lstm.build_model()
es_lstm.compile_model()

print("Model summary:\n")
print(es_lstm.get_model().summary())

Model summary:



Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 100, 80)        │     3,690,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 100, 256)       │       214,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 100, 19)        │         4,883 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,909,619 (14.91 MB)

 Trainable params: 3,909,619 (14.91 MB)

 Non-trainable params: 0 (0.00 B)

None


In [18]:
trainer = Trainer(training_config, es_lstm, preprocessor_es)

In [19]:
# Train the model
print("Training model...\n")
trainer.train((X_train_es, y_train_es), (X_dev_es, y_dev_es))
print("Training completed.\n")

Training model...

Epoch 1/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - _masked_accuracy: 0.4803 - loss: 1.7553
Epoch 1: val_loss improved from None to 0.34492, saving model to saved_models/BiLSTM_emb80_lstm128_drop30_ep20_bs64_pat3_lr10.keras
222/222 ━━━━━━━━━━━━━━━━━━━━ 46s 191ms/step - _masked_accuracy: 0.6780 - loss: 1.0779 - val__masked_accuracy: 0.9015 - val_loss: 0.3449
Epoch 2/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - _masked_accuracy: 0.9206 - loss: 0.2828
Epoch 2: val_loss improved from 0.34492 to 0.19685, saving model to saved_models/BiLSTM_emb80_lstm128_drop30_ep20_bs64_pat3_lr10.keras
222/222 ━━━━━━━━━━━━━━━━━━━━ 49s 222ms/step - _masked_accuracy: 0.9361 - loss: 0.2293 - val__masked_accuracy: 0.9407 - val_loss: 0.1969
Epoch 3/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step - _masked_accuracy: 0.9619 - loss: 0.1364
Epoch 3: val_loss improved from 0.19685 to 0.18073, saving model to saved_models/BiLSTM_emb80_lstm128_drop30_ep20_bs64_pat3_lr10.keras
222/222 ━━━━━━━━━━━

In [20]:
from evaluator.evaluator import Evaluator

evaluator_es = Evaluator(es_lstm, preprocessor_es)

In [21]:
test_metrics_es = evaluator_es.evaluate(X_test_es, y_test_es, "Test")


Test Set Evaluation:
Accuracy: 0.9388

Detailed Classification Report:
              precision    recall  f1-score   support

         ADJ       0.86      0.86      0.86       665
         ADP       0.99      0.99      0.99      1876
         ADV       0.98      0.90      0.94       424
         AUX       0.94      0.95      0.94       331
       CCONJ       0.98      0.90      0.94       395
         DET       0.98      0.99      0.99      1696
        INTJ       0.00      0.00      0.00         1
        NOUN       0.93      0.93      0.93      2225
         NUM       0.97      0.85      0.91       230
        PART       0.00      0.00      0.00         1
        PRON       0.92      0.93      0.92       445
       PROPN       0.75      0.92      0.82       818
       PUNCT       1.00      1.00      1.00      1260
       SCONJ       0.88      0.91      0.90       336
         SYM       1.00      0.76      0.86        25
        VERB       0.96      0.88      0.92      1167
         

### Inference on new sentences

In [22]:
from inference.predictor import Predictor

# Create predictor for inference
predictor_es = Predictor(es_lstm.get_model(), preprocessor_es)

In [38]:
test_sentences_spanish = {
    "NOUN vs. VERB Ambiguity": [
        "El vino estaba bueno .",            # vino => NOUN
        "Él no vino a la fiesta .",          # vino => VERB
        "Yo canto en el coro .",             # canto => VERB
        "El canto del pájaro es bonito ."   # canto => NOUN
    ],
    "NOUN vs. PROPN Ambiguity": [
        "Compró una rosa roja .",            # rosa => NOUN
        "Habló con Rosa esta mañana .",     # Rosa => PROPN
        "El marco de la foto es de madera .", # marco => NOUN
        "Vi a Marco en el parque ."         # Marco => PROPN
    ],
    "Multi-way Ambiguity (ADJ/NOUN/ADV)": [
        "El hombre bajo se fue .",           # bajo => ADJ
        "Él toca el bajo .",                 # bajo => NOUN
        "Por favor habla más bajo ."        # bajo => ADV
    ]
}

for category, sentences in test_sentences_spanish.items():
    print(f"\n--- {category} ---")
    for sentence in sentences:
        predicted_tags = predictor_es.predict_sentence(sentence)
        print(f"  Sentence: {sentence}")
        print(f"  Tags:     {' '.join(predicted_tags)}")


--- NOUN vs. VERB Ambiguity ---
  Sentence: El vino estaba bueno .
  Tags:     DET NOUN AUX ADJ PUNCT
  Sentence: Él no vino a la fiesta .
  Tags:     PRON ADV VERB ADP DET NOUN PUNCT
  Sentence: Yo canto en el coro .
  Tags:     PRON NOUN ADP DET NOUN PUNCT
  Sentence: El canto del pájaro es bonito .
  Tags:     DET NOUN ADP PROPN AUX ADJ PUNCT

--- NOUN vs. PROPN Ambiguity ---
  Sentence: Compró una rosa roja .
  Tags:     VERB DET ADJ ADJ PUNCT
  Sentence: Habló con Rosa esta mañana .
  Tags:     VERB ADP PROPN DET NOUN PUNCT
  Sentence: El marco de la foto es de madera .
  Tags:     DET NOUN ADP DET NOUN VERB ADP NOUN PUNCT
  Sentence: Vi a Marco en el parque .
  Tags:     VERB ADP PROPN ADP DET NOUN PUNCT

--- Multi-way Ambiguity (ADJ/NOUN/ADV) ---
  Sentence: El hombre bajo se fue .
  Tags:     DET NOUN ADJ PRON VERB PUNCT
  Sentence: Él toca el bajo .
  Tags:     PRON VERB DET NOUN PUNCT
  Sentence: Por favor habla más bajo .
  Tags:     ADP NOUN VERB ADV ADJ PUNCT


## German

### Preprocess the data

In [24]:
from utils import load_data

train_data_ge, dev_data_ge, test_data_ge = load_data("german")

In [25]:
preprocessor_ge = DataPreprocessor(preprocessor_config)

In [26]:
X_train_ge, y_train_ge = preprocessor_ge.process_data_to_pad_sequences(
    train_data_ge, is_train_dataset=True
)
X_dev_ge, y_dev_ge = preprocessor_ge.process_data_to_pad_sequences(
    dev_data_ge, is_train_dataset=False
)
X_test_ge, y_test_ge = preprocessor_ge.process_data_to_pad_sequences(
    test_data_ge, is_train_dataset=False
)

Sample sentence: Sehr gute Beratung , schnelle Behebung der Probleme , so stelle ich mir Kundenservice vor .
Sample tags: ADV ADJ NOUN PUNCT ADJ NOUN DET NOUN PUNCT ADV VERB PRON PRON NOUN ADP PUNCT
Sample vectorized sentence shape: (100,)
Sample vectorized sentence: [  519   405   558     3  3535 46275     4   907     3    78  9386    84
   250  4651    58     2     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
Sample vectorized tags: [ 3  1  8 13  1  8  6  8 13  3 16 11 11  8  2 13  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0 

### Initilize the model with the config

In [27]:
from models.lstm_model import LSTMModel

ge_lstm = LSTMModel(
    model_config,
    preprocessor_ge.vocab_size,
    preprocessor_ge.num_tags,
    preprocessor_config.max_sequence_length,
)
ge_lstm.build_model()
ge_lstm.compile_model()

print("Model summary:\n")
print(ge_lstm.get_model().summary())

Model summary:



Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 100, 80)        │     3,960,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 100, 256)       │       214,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 100, 19)        │         4,883 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,178,979 (15.94 MB)

 Trainable params: 4,178,979 (15.94 MB)

 Non-trainable params: 0 (0.00 B)

None


In [28]:
trainer_ge = Trainer(training_config, ge_lstm, preprocessor_ge)

In [29]:
# Train the model
print("Training model...\n")
trainer_ge.train((X_train_ge, y_train_ge), (X_dev_ge, y_dev_ge))
print("Training completed.\n")

Training model...

Epoch 1/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - _masked_accuracy: 0.4175 - loss: 1.9212
Epoch 1: val_loss improved from None to 0.51502, saving model to saved_models/BiLSTM_emb80_lstm128_drop30_ep20_bs64_pat3_lr10.keras
216/216 ━━━━━━━━━━━━━━━━━━━━ 48s 206ms/step - _masked_accuracy: 0.6087 - loss: 1.2533 - val__masked_accuracy: 0.8422 - val_loss: 0.5150
Epoch 2/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step - _masked_accuracy: 0.8915 - loss: 0.3551
Epoch 2: val_loss improved from 0.51502 to 0.28837, saving model to saved_models/BiLSTM_emb80_lstm128_drop30_ep20_bs64_pat3_lr10.keras
216/216 ━━━━━━━━━━━━━━━━━━━━ 50s 233ms/step - _masked_accuracy: 0.9187 - loss: 0.2763 - val__masked_accuracy: 0.9088 - val_loss: 0.2884
Epoch 3/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - _masked_accuracy: 0.9620 - loss: 0.1414
Epoch 3: val_loss did not improve from 0.28837
216/216 ━━━━━━━━━━━━━━━━━━━━ 50s 230ms/step - _masked_accuracy: 0.9676 - loss: 0.1206 - val__masked_accurac

In [30]:
from evaluator.evaluator import Evaluator

evaluator_ge = Evaluator(ge_lstm, preprocessor_ge)

In [31]:
test_metrics_ge = evaluator_ge.evaluate(X_test_ge, y_test_ge, "Test")


Test Set Evaluation:
Accuracy: 0.9173

Detailed Classification Report:
              precision    recall  f1-score   support

         ADJ       0.92      0.76      0.83      1249
         ADP       0.98      0.98      0.98      1603
         ADV       0.93      0.91      0.92      1058
         AUX       0.91      0.97      0.94       691
       CCONJ       0.99      0.94      0.96       460
         DET       0.99      0.98      0.98      2264
        INTJ       0.00      0.00      0.00         4
        NOUN       0.88      0.89      0.89      3111
         NUM       0.99      0.88      0.93       233
        PART       0.96      0.96      0.96       210
        PRON       0.95      0.97      0.96       705
       PROPN       0.62      0.87      0.72      1022
       PUNCT       1.00      1.00      1.00      2366
       SCONJ       0.92      0.82      0.87       168
         SYM       0.00      0.00      0.00         4
        VERB       0.93      0.80      0.86      1326
         

### Inference on new sentences

In [32]:
from inference.predictor import Predictor

# Create predictor for inference
predictor_ge = Predictor(ge_lstm.get_model(), preprocessor_ge)

In [39]:
test_sentences_german = {
    "VERB vs. NOUN (Capitalization Ambiguity)": [
        "Wir essen jetzt .",                 # essen => VERB
        "Das Essen ist fertig .",            # Essen => NOUN
        "Die Vögel fliegen .",               # fliegen => VERB
        "Die Fliegen sind nervig ."          # Fliegen => NOUN
    ],
    "Homograph Ambiguity (ADJ/NOUN/ADV)": [
        "Der Junge ist arm .",               # arm => ADJ
        "Sein Arm ist gebrochen .",          # Arm => NOUN
        "Ich komme morgen .",                # morgen => ADV
        "Der Morgen ist kalt ."              # Morgen => NOUN
    ],
    "NOUN vs. PROPN (Context Ambiguity)": [
        "Der Wolf ist im Wald .",            # Wolf => NOUN
        "Das ist Herr Wolf .",               # Wolf => PROPN
        "Der Sommer ist heiß .",             # Sommer => NOUN
        "Ich habe Frau Sommer gesehen ."     # Sommer => PROPN
    ]
}

for category, sentences in test_sentences_german.items():
    print(f"\n--- {category} ---")
    for sentence in sentences:
        predicted_tags = predictor_ge.predict_sentence(sentence)
        print(f"  Sentence: {sentence}")
        print(f"  Tags:     {' '.join(predicted_tags)}")


--- VERB vs. NOUN (Capitalization Ambiguity) ---
  Sentence: Wir essen jetzt .
  Tags:     PRON VERB ADV PUNCT
  Sentence: Das Essen ist fertig .
  Tags:     DET NOUN AUX ADJ PUNCT
  Sentence: Die Vögel fliegen .
  Tags:     DET NOUN VERB PUNCT
  Sentence: Die Fliegen sind nervig .
  Tags:     DET NOUN AUX NOUN PUNCT

--- Homograph Ambiguity (ADJ/NOUN/ADV) ---
  Sentence: Der Junge ist arm .
  Tags:     DET NOUN AUX ADJ PUNCT
  Sentence: Sein Arm ist gebrochen .
  Tags:     DET NOUN AUX VERB PUNCT
  Sentence: Ich komme morgen .
  Tags:     PRON VERB ADV PUNCT
  Sentence: Der Morgen ist kalt .
  Tags:     DET NOUN AUX ADJ PUNCT

--- NOUN vs. PROPN (Context Ambiguity) ---
  Sentence: Der Wolf ist im Wald .
  Tags:     DET NOUN AUX PROPN NOUN PUNCT
  Sentence: Das ist Herr Wolf .
  Tags:     DET AUX NOUN PROPN PUNCT
  Sentence: Der Sommer ist heiß .
  Tags:     DET NOUN AUX ADJ PUNCT
  Sentence: Ich habe Frau Sommer gesehen .
  Tags:     PRON AUX NOUN NOUN VERB PUNCT
